In [53]:
import pandas as pd
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.primitives import StatevectorSampler
from scipy.optimize import minimize
from qiskit.primitives import Sampler
from qiskit.circuit.library import TwoLocal

In [54]:
# Step 1: Load data from CSV file
file_path = 'SE1.csv'  
data = pd.read_csv(file_path)

# Step 2: Display the DataFrame 
print(data)

     Ticker       Open       High        Low      Close  Adj Close  Volume
0      ZDEU  60.189999  60.189999  60.189999  60.189999  53.323559     100
1      ZDEU  60.459999  60.459999  60.380001  60.419998  53.529995   19900
2      ZDEU  60.279999  60.279999  60.009998  60.189999  53.326221   26300
3      ZDEU  60.090000  60.090000  59.910000  59.919998  53.087009   29800
4      ZDEU  59.240002  59.240002  59.240002  59.240002  52.484554    1000
...     ...        ...        ...        ...        ...        ...     ...
1540   ZYXI   0.340000   0.340000   0.310000   0.330000   0.179540   12500
1541   ZYXI   0.290000   0.290000   0.290000   0.290000   0.157778     500
1542   ZYXI   0.290000   0.340000   0.290000   0.290000   0.157778    7300
1543   ZYXI   0.340000   0.340000   0.340000   0.340000   0.184981     600
1544   ZYXI   0.340000   0.340000   0.290000   0.290000   0.157778    1700

[1545 rows x 7 columns]


In [60]:

returns = {}
for ticker in data['Ticker'].unique():
    # Filter data for the specific ticker
    ticker_data = data[data['Ticker'] == ticker].copy()  
    
   
    if 'Adj Close' in ticker_data.columns and len(ticker_data) > 1:
       
        ticker_data['Return'] = ticker_data['Adj Close'].pct_change()
        
        
        returns[ticker] = ticker_data['Return'].dropna().values
    else:
        print(f"Not enough data for ticker: {ticker}")

print("Calculated Returns:")
for ticker, return_values in returns.items():
    print(f"{ticker}: {return_values}")


Calculated Returns:
ZDEU: [ 3.87138733e-03 -3.80671603e-03 -4.48582392e-03 -1.13484475e-02
  2.58271407e-02 -1.00378677e-02 -1.19682103e-02 -8.24355947e-03
 -3.90149643e-03  5.44965630e-03 -1.08402368e-02  1.19873891e-03
  4.95969003e-03  4.08440526e-03 -1.47457777e-02 -1.37614963e-03
 -5.59861708e-02  0.00000000e+00 -1.11315657e-02  9.41151603e-03
  9.68915916e-03 -9.59618024e-03  2.59597397e-02 -6.05836007e-03
 -7.17117189e-03  2.09463145e-02 -1.18500368e-02 -4.29575669e-03
  3.05593913e-03  1.25451697e-03 -1.79102323e-04 -2.68531918e-03
  3.76963139e-03 -5.36441446e-04 -3.45322548e-02 -3.70564510e-04
 -9.82559763e-03 -4.86815556e-03 -1.77610269e-02 -3.06477822e-04
 -1.01742030e-02 -3.34919093e-03 -2.67255914e-02  2.39489979e-03
 -2.78734574e-03 -2.17608406e-02  4.89795174e-03  2.55687147e-02
  1.13268540e-02  8.98749618e-03 -5.23961674e-03  1.28755170e-02
  1.16332644e-02 -3.90303328e-03 -7.43520859e-03  8.66554075e-03
 -9.54355860e-04  1.91102898e-03  4.95882024e-03 -1.04383566e-02

## Calculation of mean and covariance matrix 

In [63]:
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter


returns_df = pd.DataFrame.from_dict(returns, orient='index').T


covariance_matrix = returns_df.cov()


print("Covariance Matrix:")
print(covariance_matrix)

scaled_cov_matrix = covariance_matrix.values / np.max(covariance_matrix.values)


num_qubits = len(scaled_cov_matrix)
qc = QuantumCircuit(num_qubits)

params = [Parameter(f'θ{i}') for i in range(num_qubits)]


for i in range(num_qubits):
    qc.rx(params[i], i)  

print("\nQuantum Circuit:")
print(qc)


Covariance Matrix:
          ZDEU      ZGBR      XLSR       XLU       ZTR       ZTS      ZYXI
ZDEU  0.000158 -0.000003 -0.000004 -0.000021  0.000008  0.000023  0.000016
ZGBR -0.000003  0.000106 -0.000005  0.000007  0.000007  0.000003 -0.000038
XLSR -0.000004 -0.000005  0.000370 -0.000007  0.000005 -0.000019  0.000064
XLU  -0.000021  0.000007 -0.000007  0.000148  0.000006  0.000002  0.000006
ZTR   0.000008  0.000007  0.000005  0.000006  0.000135 -0.000010 -0.000056
ZTS   0.000023  0.000003 -0.000019  0.000002 -0.000010  0.000191  0.000041
ZYXI  0.000016 -0.000038  0.000064  0.000006 -0.000056  0.000041  0.004881

Quantum Circuit:
     ┌────────┐
q_0: ┤ Rx(θ0) ├
     ├────────┤
q_1: ┤ Rx(θ1) ├
     ├────────┤
q_2: ┤ Rx(θ2) ├
     ├────────┤
q_3: ┤ Rx(θ3) ├
     ├────────┤
q_4: ┤ Rx(θ4) ├
     ├────────┤
q_5: ┤ Rx(θ5) ├
     ├────────┤
q_6: ┤ Rx(θ6) ├
     └────────┘


In [64]:
def objective_function(weights, mean_returns, covariance_matrix, risk_free_rate=0.01):
    # Calculate portfolio return
    portfolio_return = np.dot(weights, mean_returns)
    
    # Calculate portfolio variance
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    
    # Calculate portfolio standard deviation
    portfolio_std_dev = np.sqrt(portfolio_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    
    return -sharpe_ratio  # Negative because we will minimize this in optimization


In [62]:
import numpy as np
import pandas as pd

# Step 1: Calculate mean returns
mean_returns = {ticker: np.mean(return_values) for ticker, return_values in returns.items()}

# Step 2: Create a DataFrame from the returns dictionary
returns_df = pd.DataFrame.from_dict(returns, orient='index').T

# Step 3: Calculate the covariance matrix
covariance_matrix = returns_df.cov()

# Optional: Display the results
print("Mean Returns:")
print(mean_returns)

print("\nCovariance Matrix:")
print(covariance_matrix)



Mean Returns:
{'ZDEU': np.float64(-0.00033247359866931823), 'ZGBR': np.float64(-0.00022644429328985016), 'XLSR': np.float64(-0.00015091704877137244), 'XLU': np.float64(2.4477439256854356e-05), 'ZTR': np.float64(0.00018521656769896058), 'ZTS': np.float64(0.00018489503223608958), 'ZYXI': np.float64(-0.006938322158227981)}

Covariance Matrix:
          ZDEU      ZGBR      XLSR       XLU       ZTR       ZTS      ZYXI
ZDEU  0.000158 -0.000003 -0.000004 -0.000021  0.000008  0.000023  0.000016
ZGBR -0.000003  0.000106 -0.000005  0.000007  0.000007  0.000003 -0.000038
XLSR -0.000004 -0.000005  0.000370 -0.000007  0.000005 -0.000019  0.000064
XLU  -0.000021  0.000007 -0.000007  0.000148  0.000006  0.000002  0.000006
ZTR   0.000008  0.000007  0.000005  0.000006  0.000135 -0.000010 -0.000056
ZTS   0.000023  0.000003 -0.000019  0.000002 -0.000010  0.000191  0.000041
ZYXI  0.000016 -0.000038  0.000064  0.000006 -0.000056  0.000041  0.004881


In [72]:
import numpy as np

def objective_function(weights, mean_returns, covariance_matrix, risk_free_rate=0.01):
    # Calculate portfolio return
    portfolio_return = np.dot(weights, mean_returns)
    
    # Calculate portfolio variance
    portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
    
    # Calculate portfolio standard deviation
    portfolio_std_dev = np.sqrt(portfolio_variance)
    
    # Calculate Sharpe Ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    
    # Return the negative Sharpe Ratio and the other metrics for output
    return -sharpe_ratio, portfolio_return, portfolio_variance, portfolio_std_dev

# Example weights and means (replace these with actual values)
weights = np.array([0.5, 0.3, 0.2, 0.1, 0.4, 0.2, 0.1])  
mean_returns = np.array([-0.0003324, -0.0002264, -0.000150, 2.4474, 0.000185, 0.000184, -0.00693])  

covariance_matrix = np.array([
    [0.000158, -0.000003, -0.000004, -0.000021, 0.000008, 0.000023, 0.000016],
    [-0.000003, 0.000106, -0.000005, 0.000007, 0.000007, 0.000003, -0.000038],
    [-0.000004, -0.000005, 0.000370, -0.000007, 0.000005, -0.000019, 0.000064],
    [-0.000021, 0.000007, -0.000007, 0.000148, 0.000006, 0.000002, 0.000006],
    [0.000008, 0.000007, 0.000005, 0.000006, 0.000135, -0.000010, -0.000056],
    [0.000023, 0.000003, -0.000019, 0.000002, -0.000010, 0.000191, 0.000041],
    [0.000016, -0.000038, 0.000064, 0.000006, -0.000056, 0.000041, 0.004881]
])  

# Calculate objective metrics
neg_sharpe_ratio, portfolio_return, portfolio_variance, portfolio_std_dev = objective_function(weights, mean_returns, covariance_matrix)

# Output the results
print("Negative Sharpe Ratio:", neg_sharpe_ratio)
print("Portfolio Return:", portfolio_return)
print("Portfolio Variance:", portfolio_variance)
print("Portfolio Standard Deviation:", portfolio_std_dev)


Negative Sharpe Ratio: -19.334018222683802
Portfolio Return: 0.24389368
Portfolio Variance: 0.00014635
Portfolio Standard Deviation: 0.012097520407091694
